In [390]:
import numpy as np

In [391]:
class ConvoyCandidate(object):
    """
    Attributes:
        indices(set): The object indices assigned to the convoy
        is_assigned (bool):
        start_time (int):  The start index of the convoy
        end_time (int):  The last index of the convoy
    """
    __slots__ = ('indices', 'is_assigned', 'start_time', 'end_time')

    def __init__(self, indices, is_assigned, start_time, end_time):
        self.indices = indices
        self.is_assigned = is_assigned
        self.start_time = start_time
        self.end_time = end_time

    def __repr__(self):
        return '<%r %r indices=%r, is_assigned=%r, start_time=%r, end_time=%r>' % (self.__class__.__name__, id(self), self.indices, self.is_assigned, self.start_time, self.end_time)


class CMC(object):
    """
    An implementation of the Coherence Moving Cluster (CMC) algorithm

    Attributes:
        k (int):  Minimum number of consecutive timestamps to be considered a convoy
        m (int):  Minimum number of members to be considered a convoy

    Original paper: Discovery of Convoys in Trajectory Databases
    https://arxiv.org/pdf/1002.0963.pdf
    """
    def __init__(self, clf, k, m):
        self.clf = clf
        self.k = k
        self.m = m

    def fit_predict(self, X, y=None, sample_weight=None):
        convoy_candidates = set()
        columns = len(X[0])
        column_iterator = range(columns)
        output_convoys = []

        for column in column_iterator:
            current_convoy_candidates = set()
            values = [row[column] if isinstance(row[column], (list, set)) else [row[column]] for row in X]
            if len(values) < self.m:
                continue
            clusters = self.clf.fit_predict(values, y=y, sample_weight=sample_weight)
            unique_clusters = set(clusters)
            clusters_indices = dict((cluster, ConvoyCandidate(indices=set(), is_assigned=False, start_time=None, end_time=None)) for cluster in unique_clusters)

            for index, cluster_assignment in enumerate(clusters):
                clusters_indices[cluster_assignment].indices.add(index)

            # update existing convoys
            for convoy_candidate in convoy_candidates:
                convoy_candidate_indices = convoy_candidate.indices
                convoy_candidate.is_assigned = False
                for cluster in unique_clusters:
                    cluster_indices = clusters_indices[cluster].indices
                    cluster_candidate_intersection = cluster_indices & convoy_candidate_indices
                    if len(cluster_candidate_intersection) < self.m:
                        continue
                    convoy_candidate.indices = cluster_candidate_intersection
                    current_convoy_candidates.add(convoy_candidate)
                    convoy_candidate.end_time = column
                    clusters_indices[cluster].is_assigned = convoy_candidate.is_assigned = True

                # check if candidates qualify as convoys
                candidate_life_time = (convoy_candidate.end_time - convoy_candidate.start_time) + 1
                if (not convoy_candidate.is_assigned or column == column_iterator[-1]) and candidate_life_time >= self.k:
                    output_convoys.append(convoy_candidate)

            # create new candidates
            for cluster in unique_clusters:
                cluster_data = clusters_indices[cluster]
                if cluster_data.is_assigned:
                    continue
                cluster_data.start_time = cluster_data.end_time = column
                current_convoy_candidates.add(cluster_data)
            convoy_candidates = current_convoy_candidates
        return output_convoys

In [392]:

# test_data = ([[3],[5],[6]],
#              [[4],[6],[7]],
#             [[5],[7],[49]],
#             [[51],[52],[54]])

# test_data =([[3,1],[4,2],[50,10000],[51,10001]])
# clustering = DBSCAN(eps=2,min_samples=2).fit_predict(test_data)
# print(clustering)

In [393]:
from sklearn.cluster import DBSCAN

clustering_clf = DBSCAN(eps=2,min_samples=2)
clf = CMC(clustering_clf, k=3, m=2)
# data = (
#     [[932.0], [1870.0], [2814.0], [3749.0], [4693.0], [5595.0], [6461.0], [7392.0]],
#     [[938.0], [1873.0], [2820.0], [3750.0], [4695.0], [5612.0], [6552.0], [7499.0]]
# )

# data = ([[936], [1870], [2814], [3719], [4601]],
#              [[939], [1873], [2817], [3750], [4525]],
#              [[943], [1877], [2825], [3741], [4236]],
#              [[946], [1879], [2826], [3762], [4947]])

# test_data = ([[1], [2], [3], [5]],
#              [[4], [7], [7],[10]])

# data = ([[1],[3],[5],[7],[13],[17]], 
#              [[4],[6],[8],[12],[14],[15]],
#             [[5],[7],[9],[11],[16],[19]],
#             [[10],[12],[13],[21],[26],[29]])
data = ([[3,1,3],[3,2,4],[2,4,5]],
        [[4,2,2],[6,2,3],[4,4,5]],
        [[1,1,3],[51,2,-1],[55,3,0]],
        [[51,2,1],[52,2,4],[56,-3, 2]])


# data = ([[3,1],[4,2],[5,1]],
#         [[3,2],[6,2],[5,3]],
#         [[1,1],[25,4],[54,3]],
#         [[51,1],[51,2],[56,3]])
# data = (
#     [[932.0], [1870.0], [2814.0], [3749.0], [4693.0], [5595.0], [6461.0], [7392.0]],
#     [[938.0], [1873.0], [2820.0], [3750.0], [4695.0], [5612.0], [6552.0], [7499.0]])

convoys = clf.fit_predict(data)
print (convoys)
for convoy in convoys:
    print('Convoy')
    for i in convoy.indices:
        print('%i: %r - Start Time: %r, End Time: %r' % (i, data[i], convoy.start_time, convoy.end_time))

[<'ConvoyCandidate' 2393603919936 indices={0, 1}, is_assigned=True, start_time=0, end_time=2>]
Convoy
0: [[3, 1], [4, 2], [5, 1]] - Start Time: 0, End Time: 2
1: [[3, 2], [6, 2], [5, 3]] - Start Time: 0, End Time: 2
